# Definitions (run first!)

In [69]:
!pip install wandb
!wandb login

     |████████████████████████████████| 1.8 MB 1.6 MB/s 
     |████████████████████████████████| 131 kB 10.7 MB/s 
  Using cached subprocess32-3.5.4-py3-none-any.whl
  Using cached PyYAML-5.4.1-cp38-cp38-manylinux1_x86_64.whl (662 kB)
  Using cached shortuuid-1.0.1-py3-none-any.whl (7.5 kB)
     |████████████████████████████████| 159 kB 10.1 MB/s 
  Using cached promise-2.3-py3-none-any.whl
  Using cached psutil-5.8.0-cp38-cp38-manylinux2010_x86_64.whl (296 kB)
  Using cached pathtools-0.1.2-py3-none-any.whl
  Using cached docker_pycreds-0.4.0-py2.py3-none-any.whl (9.0 kB)
     |████████████████████████████████| 63 kB 4.7 MB/s 
wandb: Currently logged in as: ut-mit-news-classify (use `wandb login --relogin` to force relogin)


In [2]:
import gzip
import pickle
import random
import requests
import csv
from torch.utils.data import Dataset, DataLoader, random_split
from mitnewsclassify.gpt_model import GPTModel as GPTHead
from transformers import GPT2Tokenizer, GPT2Model
import torch
from sklearn.preprocessing import MultiLabelBinarizer
import numpy as np
import wandb

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
seed = 42

In [4]:
class GPTEmbeddedDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx], idx

In [5]:
import csv

def loadcsv(filename):
    with open(filename, newline='', encoding='utf-8') as f:
        return list(csv.reader(f))

def load_label_map(out2id_path, id2label_path):
    
    out2id = loadcsv(out2id_path)
    out2id = {int(row[0]): row[1] for row in out2id}

    id2label_raw = loadcsv(id2label_path)
    id2label = {}

    for row in id2label_raw:
        if row == []:
            continue
        id2label[row[1]] = row[2]

    out2label = [id2label[out2id[out]] for out in sorted(out2id.keys())]
    
    return out2label

out2label = load_label_map('labels_dict_gpt.csv', 'nyt-theme-tags.csv')
mlb = MultiLabelBinarizer(classes=out2label)
mlb.fit(out2label)

MultiLabelBinarizer(classes=['suspensions, dismissals and resignations',
                             'education and schools',
                             'colleges and universities', 'blacks',
                             'population', 'economic conditions and trends',
                             'labor',
                             'office buildings and commercial properties',
                             'architecture', 'medicine and health',
                             'awards, decorations and honors',
                             'diseases and conditions', 'research', 'cancer',
                             'basketball', 'design', 'interior design',
                             'real estate', 'trades (sports)',
                             'demonstrations and riots', 'dancing',
                             'hockey, ice', 'games', 'playoff games',
                             'baseball', 'travel and vacations', 'finances',
                             'books and literature',
   

In [6]:
# temporary dataset for storing tokenized articles & transformed labels
class NYTDataset(Dataset):
    def __init__(self, articles, labels):

        self.tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
        self.tokenizer.pad_token = self.tokenizer.eos_token

        print('Tokenizing...')
        self.articles = self.tokenizer(articles, add_special_tokens=True, padding="max_length", truncation=True,
                                       max_length=1024, return_tensors="pt", return_attention_mask=True)

        self.input_ids = self.articles['input_ids']
        self.attention_mask = self.articles['attention_mask']

        print('Preprocessing labels...')
        self.labels = mlb.transform(labels)
        print('Done')

    def __len__(self):
        return len(self.articles)

    def __getitem__(self, idx):
        # return self.articles[idx], self.labels[idx]
        return self.input_ids[idx], self.attention_mask[idx], self.labels[idx]

In [7]:
import time
import math

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

def validation_split(dataset, validation_subset, seed=42):

    if validation_subset > 0:
        n_total_samples = len(dataset)
        n_train_samples = math.floor(n_total_samples * (1-validation_subset))
        n_valid_samples = n_total_samples - n_train_samples

        train_subset, valid_subset = random_split(
            dataset,
            [n_train_samples, n_valid_samples],
            generator=torch.Generator().manual_seed(seed)
        )  # reproducible results

    else:
        train_subset = dataset
        valid_subset = None

    return train_subset, valid_subset

In [8]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    epoch_precision = 0
    epoch_recall = 0
    epoch_f_score = 0
    
    model.train()
    

    for i, batch in enumerate(iterator):

        article_embeddings, labels, idx  = batch
        article_embeddings = article_embeddings.to(device)
        labels = labels.type(torch.float).to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(article_embeddings)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # calculate metrics
        preds = model.act(outputs) > 0.5

        acc, precision, recall, f1 = multi_label_scores(labels.detach().cpu(), preds.detach().cpu())
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        epoch_precision += precision.item()
        epoch_recall += recall.item()
        epoch_f_score += f1.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator), \
        epoch_precision / len(iterator), epoch_recall / len(iterator), \
        epoch_f_score / len(iterator)

In [9]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    epoch_precision = 0
    epoch_recall = 0
    epoch_f_score = 0
    
    model.eval()
    
    with torch.no_grad():
        for i, batch in enumerate(iterator):

            article_embeddings, labels, idx  = batch
            article_embeddings = article_embeddings.to(device)
            labels = labels.type(torch.float).to(device)

            outputs = model(article_embeddings)
            loss = criterion(outputs, labels)

            # calculate metrics
            preds = model.act(outputs) > 0.5

            acc, precision, recall, f1 = multi_label_scores(labels.detach().cpu(), preds.detach().cpu())
            epoch_loss += loss.item()
            epoch_acc += acc.item()
            epoch_precision += precision.item()
            epoch_recall += recall.item()
            epoch_f_score += f1.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator), \
        epoch_precision / len(iterator), epoch_recall / len(iterator), \
        epoch_f_score / len(iterator)

# Training

In [10]:
import io
import os
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from mitnewsclassify.gpt_model import GPTModel as GPTHead
from tqdm.notebook import tqdm

%load_ext autoreload
%autoreload 2

In [11]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

def multi_label_scores(correct_labels, predicted_labels):

    accuracy = accuracy_score(correct_labels, predicted_labels)
    precision = precision_score(correct_labels, predicted_labels, average='weighted', zero_division=0)
    recall = recall_score(correct_labels, predicted_labels, average='weighted', zero_division=0)
    f_1_score = f1_score(correct_labels, predicted_labels, average='weighted', zero_division=0)
    
    return accuracy, precision, recall, f_1_score

def gettags(head_model, features, eval=False):
    head_model.eval()
    features = features.unsqueeze(0).to(device)
    with torch.no_grad():
        logits = head_model(features)
        multi_label_sigmoids = head_model.act(logits)

    preds = multi_label_sigmoids > 0.5
    preds = preds.detach().cpu()

    return mlb.inverse_transform(preds)

In [12]:
train_dataset = torch.load('embedded_train_100k_baseline.pt')

print('X_train', train_dataset.X.shape)
print('y_train', train_dataset.y.shape)

X_train torch.Size([100000, 768])
y_train torch.Size([100000, 538])
X_test torch.Size([10000, 768])
y_test torch.Size([10000, 538])


In [14]:
# splitting train/validation
batch_size = 128

train_subset, valid_subset = validation_split(train_dataset, 0.1, seed)
train_loader = DataLoader(train_subset, batch_size=batch_size)
valid_loader = DataLoader(valid_subset, batch_size=batch_size)

n_training_samples = train_dataset.X.shape[0]

In [15]:
from mitnewsclassify.gpt_model import GPTModel as GPTHead

In [13]:
wandb.init(
    entity='ut-mit-news-classify',
    project="NYT Multilabeling"
)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: ut-mit-news-classify (use `wandb login --relogin` to force relogin)


In [18]:
%%time
# hyperparams
max_epochs = 1000

patience = 10

# model
model = GPTHead(768, 538).to(device)
optimizer = optim.Adam(model.parameters(),
                    lr = 5e-1, # default is 5e-5, our notebook had 2e-5
                  )

# training
epochs_of_no_improvement = 0
best_valid_loss = float('inf')

wandb.config.early_stopping_patience = patience
wandb.config.training_samples=n_training_samples

model_file_name = f'model_{n_training_samples//1000}K_samples_baseline.pt'

for epoch in range(max_epochs):

    start_time = time.time()
    
    train_loss, train_acc, train_precision, train_recall, train_f_score \
        = train(model, train_loader, optimizer, criterion)
    valid_loss, valid_acc, valid_precision, valid_recall, valid_f_score \
        = evaluate(model, valid_loader, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        print(f'New validation loss {valid_loss} is better than the best validation loss {best_valid_loss} so far.')
        best_valid_loss = valid_loss
        torch.save(model, model_file_name)
        epochs_of_no_improvement = 0
    else: 
        epochs_of_no_improvement += 1
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}% | ' +
          f'Train Precision: {train_precision*100:.2f}% | Train Recall: {train_recall*100:.2f}% | ' +
          f'Train F1-score: {train_f_score*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}% | ' +
          f'Val. Precision: {valid_precision*100:.2f}% | Val. Recall: {valid_recall*100:.2f}% | ' +
          f'Val. F1-score: {valid_f_score*100:.2f}%')
    
    wandb.log({"train_loss": train_loss, 
                "train_precision": train_precision, 
                "train_f_score": train_f_score, 
                "train_acc": train_acc,
                "train_recall": train_recall,
               "valid_loss": valid_loss,
               "valid_acc": valid_acc,
               "valid_precision": valid_precision,
               "valid_recall": valid_recall,
               "valid_f_score": valid_f_score,
               "epoch": epoch+1,
                })
    # check if the training should be stopped and then stop the training
    if epochs_of_no_improvement == patience : 
        print(f'Early stopping, on epoch: {epoch+1}.')
        break

torch.cuda.empty_cache()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: ut-mit-news-classify (use `wandb login --relogin` to force relogin)


New validation loss 0.02481998378245891 is better than the best validation loss inf so far.
Epoch: 01 | Epoch Time: 0m 17s
	Train Loss: 0.031 | Train Acc: 5.60% | Train Precision: 8.81% | Train Recall: 5.99% | Train F1-score: 6.60%
	 Val. Loss: 0.025 |  Val. Acc: 7.95% | Val. Precision: 15.24% | Val. Recall: 9.48% | Val. F1-score: 10.95%
Epoch: 02 | Epoch Time: 0m 16s
	Train Loss: 0.024 | Train Acc: 10.59% | Train Precision: 20.36% | Train Recall: 13.24% | Train F1-score: 14.94%
	 Val. Loss: 0.027 |  Val. Acc: 5.59% | Val. Precision: 16.39% | Val. Recall: 9.40% | Val. F1-score: 10.12%
Epoch: 03 | Epoch Time: 0m 17s
	Train Loss: 0.022 | Train Acc: 12.89% | Train Precision: 24.89% | Train Recall: 16.37% | Train F1-score: 18.49%
	 Val. Loss: 0.035 |  Val. Acc: 6.89% | Val. Precision: 13.92% | Val. Recall: 7.28% | Val. F1-score: 6.51%
Epoch: 04 | Epoch Time: 0m 17s
	Train Loss: 0.022 | Train Acc: 14.16% | Train Precision: 28.20% | Train Recall: 18.48% | Train F1-score: 20.91%
	 Val. Loss: 

train_loss,0.01829
train_precision,0.4072
train_f_score,0.31126
train_acc,0.20457
train_recall,0.27621
valid_loss,0.03027
valid_acc,0.08426
valid_precision,0.21634
valid_recall,0.09864
valid_f_score,0.11279
epoch,19


train_loss,█▄▃▃▃▂▂▂▂▂▂▁▁▂▁▁▁▁▁
train_precision,▁▄▅▅▆▆▇▇▇▇▇▇█▇█████
train_f_score,▁▃▄▅▆▆▆▇▇▇▇▇▇▇▇████
train_acc,▁▃▄▅▅▆▆▇▇▇▇▇█▇▇████
train_recall,▁▃▄▅▅▆▆▇▇▇▇▇▇▇▇▇███
valid_loss,▂▄█▃▃▂▂▂▁▃▁▃▃▂▄▃▁▃▅
valid_acc,▃▁▂▄▁▃▄▂▇▄▄▃▁▇▆▅█▅▃
valid_precision,▂▂▁▁▂▅▆▅▇▅▆▅▅▇▅▆█▆▄
valid_recall,▂▂▁▂▁▃▅▄█▅▅▅▄▇▆▄▇▄▂
valid_f_score,▃▃▁▂▂▄▆▅█▅▆▅▅▇▆▅█▅▃
epoch,▁▁▂▂▃▃▃▄▄▅▅▅▆▆▆▇▇██


CPU times: user 24min 18s, sys: 1.59 s, total: 24min 19s
Wall time: 5min 34s


In [ ]:
test_dataset = torch.load('embedded_test_100k_baseline.pt')
print('X_test', test_dataset.X.shape)
print('y_test', test_dataset.y.shape)

In [18]:
# evaluate the model on the test set

test_model = torch.load('model_100K_samples_baseline.pt')
test_loader = DataLoader(test_dataset, batch_size=64)
criterion = nn.BCEWithLogitsLoss()

start_time = time.time()
test_loss, test_acc, test_precision, test_recall, test_f_score \
    = evaluate(test_model, test_loader, criterion)
end_time = time.time()
epoch_mins, epoch_secs = epoch_time(start_time, end_time)

wandb.run.summary['test_acc'] = test_acc
wandb.run.summary['test_precision'] = test_precision
wandb.run.summary['test_recall'] = test_recall
wandb.run.summary['test_f_score'] = test_f_score

print(f'Epoch: test | Epoch Time: {epoch_mins}m {epoch_secs}s')
print(f'\tTest Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}% | ' +
      f'Test Precision: {test_precision*100:.2f}% | Test Recall: {test_recall*100:.2f}% | ' +
      f'Test F1-score: {test_f_score*100:.2f}%')

wandb.finish()

Epoch: test | Epoch Time: 0m 2s
	Test Loss: 0.023 | Test Acc: 12.98% | Test Precision: 24.81% | Test Recall: 20.86% | Test F1-score: 21.40%


test_acc,0.12978
test_precision,0.24813
test_recall,0.20862
test_f_score,0.21404


In [81]:
idx = 15
print('predicted', gettags(model, train_dataset.X[idx]))
print('gold:', mlb.inverse_transform(train_dataset.y[idx].unsqueeze(0)))

predicted [('politics and government',)]
gold: [('armament, defense and military forces',)]
